### 引言

Heikin Ashi蜡烛图是日本人在原始蜡烛图的基础上，进行修正得到的一种价格图形
日文Heikin是平均，Ashi是节奏，用来反映股价运动的平均节奏的。

HA蜡烛图和原始蜡烛图一样，每一根HA蜡烛线，也是由四个价格组成：HA开盘价，HA收盘价，HA最高价，HA最低价。但是，这些价格不是交易日当天的价格，而是根据历史价格计算出来的。

### 计算方法及基本原理

这四个价格均是根据原始蜡烛图的四个原始价格（开盘价、收盘价、最高价、最低价）计算而来的。

HA蜡烛线的四个价格计算方法如下∶  

HA收盘价ha_close=(开盘价open+收盘价close+最高价high+最低价low)/4

HA开盘价ha_open =(上一交易日HA开盘价+上一交易日HA收盘价)/2

HA最高价ha_high =MAX（最高价high,HA开盘价ha_open,HA收盘价ha_close)

HA最低价ha_low  =MIN（最低价low,HA开盘价ha_open,HA收盘价ha_close)

有时候，K线图中会发出很多噪声，这些噪声掩盖了市场真实趋势的随机波动（包括价格波动和交易量波动）。它可能是具有有限持续市场影响的基本面新闻，也可能是对技术指标和市场趋势解读造成混淆的无意义短期价格和交易量波动。由于噪声之所以成为噪声，是因为其对交易者分析市场产生了干扰和误导，对交易产生不利影响。  

正因如此，HA蜡烛图应运而生，HA中的四个价格中，HA开盘价和HA收盘价都是经过平均计算得来，平均化的处理相当于噪声消除处理，在一定程度上消除了市场的噪声，可以更加明确地反映市场价格的走势。

HA蜡烛图往往是一连串的绿色上行蜡烛线（阳烛），或者一连串的红色下行蜡烛图（阴烛），当颜色转变时（绿变红或红变绿），往往预示着价格的反转（由涨转跌或由跌转涨），当然也并非绝对是这样，有时也会出现“意外”。

### Python代码实现及可视化

In [1]:
#先引入后面可能用到的包（package）
import pandas as pd  
import numpy as np
import tushare as ts 
import matplotlib.pyplot as plt
%matplotlib inline   

#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

#### 获取数据

In [13]:
#设置token
token='e0eeb08befd1f07516df2cbf9cbd58663f77fd72f92a04f290291c9d'
pro=ts.pro_api(token)
#常用大盘指数
index={'上证综指': '000001.SH','深证成指': '399001.SZ','沪深300': '000300.SH','创业板指': '399006.SZ',
        '上证50': '000016.SH','中证500': '000905.SH','中小板指': '399005.SZ','上证180': '000010.SH'}
#获取代码
def get_code(name):
    df = pro.stock_basic(exchange='', list_status='L')
    codes=df.ts_code.values
    names=df.name.values
    stock=dict(zip(names,codes))
    #合并指数和个股成一个字典
    stocks=dict(stock,**index)
    return stocks[name]    

#默认设定时间周期为当前时间往前推300个交易日
#日期可以根据需要自己改动
def get_data(name,n=300):
    from datetime import datetime,timedelta
    t=datetime.now()
    t0=t-timedelta(n)
    start=t0.strftime('%Y%m%d')
    end=t.strftime('%Y%m%d')
    code=get_code(name)
    #如果代码在字典index里，则取的是指数数据
    if code in index.values():
        df=pro.index_daily(ts_code=code,start_date=start, end_date=end)
    #否则取的是个股数据，使用前复权
    else:
        df=ts.pro_bar(ts_code=code, adj='qfq', start_date=start, end_date=end)
    #将交易日期设置为索引值
    df.index=pd.to_datetime(df.trade_date)
    df=df.sort_index()
    return df

#### 计算Heikin Ashi蜡烛线

In [37]:
def cal_hadata(name):
    df=df=get_data(name)
    #计算修正版K线
    df['ha_close']=(df.close+df.open+df.high+df.low)/4.0
    ha_open=np.zeros(df.shape[0])
    ha_open[0]=df.open[0]
    for i in range(1,df.shape[0]):
        ha_open[i]=(ha_open[i-1]+df['ha_close'][i-1])/2
    df.insert(1,'ha_open',ha_open)
    df['ha_high']=df[['high','ha_open','ha_close']].max(axis=1)
    df['ha_low']=df[['low','ha_open','ha_close']].min(axis=1)
    df=df.iloc[1:]
    return df

#### 对K线进行可视化

In [38]:
def kline_plot(name,ktype=0):
    df=cal_hadata(name)
    #画K线图数据
    date = df.index.strftime('%Y%m%d').tolist()
    if ktype==0:
        k_value = df[['open','close', 'low', 'high']].values
    else:
        k_value = df[['ha_open','ha_close', 'ha_low', 'ha_high']].values
    #引入pyecharts画图使用的是0.5.11版本，新版命令需要重写
    from pyecharts import Kline,Line, Bar, Scatter,Overlap
    kline = Kline(name+'行情走势')
    kline.add('日K线图', date, k_value,
              is_datazoom_show=True,is_splitline_show=False)
    #加入5、20日均线
    df['ma20']=df.close.rolling(20).mean()
    df['ma5']=df.close.rolling(5).mean()
    line = Line()
    v0=df['ma5'].round(2).tolist()
    v=df['ma20'].round(2).tolist()
    line.add('5日均线', date,v0 ,is_symbol_show=False,line_width=2)
    line.add('20日均线', date,v ,is_symbol_show=False,line_width=2)
    # 成交量
    bar = Bar()
    bar.add('成交量', date, df['vol'],tooltip_tragger='axis', is_legend_show=False, 
            is_yaxis_show=False, yaxis_max=5*max(df['vol']))
    overlap = Overlap()
    overlap.add(kline)
    overlap.add(line,)
    overlap.add(bar,yaxis_index=1, is_add_yaxis=True)
    return overlap

In [39]:
kline_plot('沪深300',ktype=0)

In [40]:
kline_plot('沪深300',ktype=1)

In [43]:
kline_plot('中国平安',ktype=0)

In [45]:
kline_plot('中国平安',ktype=1)

下面两个图分别是HA蜡烛图的上涨趋势和下跌趋势，很明显，在上涨趋势中，HA蜡烛图基本都是阳烛，表示都是收涨的蜡烛线，很少会出现阴烛，且在强劲的上涨趋势中，很少会出现下影线；

相对地，在下跌趋势中，HA蜡烛图基本都是阴烛，表示都是收跌的蜡烛线，很少会出现阳烛，且在强劲的下跌趋势中，很少会出现上影线。

这些信号比传统的烛图更容易识别趋势或交易机会。图形上显示的趋势不会经常被错误信号打断，因此更容易被发现。

上面的图表示例展示了平均K线图（Heikin-Ashi）如何用于分析和做出交易决策。在图的左方，长的红色蜡烛伴随下跌，向下的影线非常短。随着价格的不断下降，向下的影线会变长，这表明价格可能会被推高，买压正在积聚。紧随其后的是强劲的上行走势。

向上的移动是强烈的，没有给出反转的主要迹象，直到有几个较小蜡烛并列在一排，两边都有影线，表明趋势不够明朗。此时，交易员可以从更宏观的角度来判断他们应该做多还是做空。

一旦趋势开始，平均K线图（Heikin-Ashi）也可以用来让交易者继续交易。通常情况下，最好是在平均K线图（Heikin-Ashi）的蜡烛变色之前一直交易。另外，颜色的变化并不总是意味着趋势的结束 - 也可能只是停顿。

平均K线图（Heikin-Ashi）有五个可以用于识别趋势和购买机会的主要信号：

1. 空心或绿色的蜡烛伴随无向下“影线”，表明强劲的上升趋势：让你的利润增长！

2. 空心或绿色的蜡烛表示上涨趋势：此时你可能应该增加多头头寸，退出空头头寸。

3. 体型小、被上下影线环绕的蜡烛预示着趋势的变化：喜欢冒险的交易员可能会在这里买进或卖出，而其他人则会等到确认信号出现后再做多或做空。

4. 实心或红色的蜡烛表示下跌趋势：此时你可能应该增持空头头寸，并退出多头头寸。

5. 实心或红色的蜡烛伴随无向上“影线”，表明强劲的下跌趋势：在趋势发生变化之前不要做多。

HA蜡烛图在使用的过程中，有如下几点规则：

● 阳烛表示趋势向上，不带下影线的阳烛信号更强烈

● 阴烛表示趋势向下，不带上影线的阴烛信号更强烈

● 烛身较小，且带上下影线，显示趋势将可能出现反转

### 结语

HA蜡烛图是一个延迟信号，这是它的优点也是它的缺点。

因为它比较滞后，所以产生的交易信号比较可靠，消除了市场噪音，让我们更容易把握趋势以及捕捉买卖机会，但也是因为滞后，所以进场不好把握。

而原始蜡烛图则最能实时反映价格变动，基本不存在滞后性，所以建议读者们还是将原始蜡烛图和HA蜡烛图结合来看。

。平均K线图（Heikin-Ashi）的图表的优势是更能明确地反映一个趋势是否开始及完结。但需要强调的是，平均K线图（Heikin-Ashi）的图表不是一定比传统的蜡烛图好，大家也不能只运用图表来作入市判断，它只是一个参考工具，可以试试套用在个人的交易策略上，看看会否能提高回报。相信不少交易者都明白，在一个急升/急跌趋势出现后，你往往不能期望在升浪/跌浪的「尽头」平仓，大多会是是希望在走势逆转前平仓的话，虽赚不尽，但每次这样做反而令交易的获利幅度提高。